In [15]:
import sys

import numpy as np
import pandas as pd
import datetime
import time

# music api imports
import spotipy
import spotipy.util as util
import pygn

import dbutils

from time import sleep


In [28]:
# Setting up tokens and authorization - hidden for github, use own tokens
# spotify
# if token expired, get it from here: https://developer.spotify.com/web-api/console/get-audio-features-several-tracks/#complete
# use these as track ids: 00UcJ7vTs2venzjRkSHzks, 00fJ7w7yVcBSdMvzV0cS41
spotify = spotipy.Spotify(auth='BQC4ixCcuiI42KsFhV6asMyO4BJZQDA3tchC2L_BN_-dda0FnehIzbBjSzV4yODL3NvQUBrsaelNhzpP0NS-7-f-Oq1n80bFnvv4UWazs_uIphY8KUKoUtUzbvGMW2lITnSabacSQ5stuO0_xD1cSOk5D6Kgimw')

# gracenote
clientid = '18404282-D215A8393036073C7D4008B567ADB278' # Enter your Client ID here
# userid = pygn.register(clientid)
userid = '91860226739687882-7862D6C2BEF5300543BC0BFE3BE6F36D'

In [17]:
# Examples of gracenote fuzzy match & full records
song1 = pygn.search(clientid, userid, artist='Zara', track='Lush')
song2 = pygn.search(clientid, userid, artist='The Chainsmokers', track='Closer')

song1

{'album_art_url': 'http://akamai-b.cdn.cddbp.net/cds/2.0/cover/0EA1/E765/D40E/462C_medium_front.jpg?cid=18404282',
 'album_artist_name': 'Zara Larsson',
 'album_gnid': '700241972-A4980D93DE119D5295F7AA2F9BB52D01',
 'album_title': 'So Good',
 'album_year': '2017',
 'artist_bio_url': '',
 'artist_era': {'1': {'ID': '42877', 'TEXT': '2010s'}},
 'artist_image_url': 'http://akamai-b.cdn.cddbp.net/cds/2.0/image-artist/7133/8F9B/C00C/4E38_medium_front.jpg?cid=18404282',
 'artist_origin': {'1': {'ID': '29896', 'TEXT': 'Scandinavia'},
  '2': {'ID': '29991', 'TEXT': 'Sweden'},
  '4': {'ID': '30894', 'TEXT': 'Stockholm'}},
 'artist_type': {'1': {'ID': '29423', 'TEXT': 'Female'},
  '2': {'ID': '29443', 'TEXT': 'Female'}},
 'genre': {'1': {'ID': '35469', 'TEXT': 'Pop'},
  '2': {'ID': '35493', 'TEXT': 'Western Pop'},
  '3': {'ID': '25637', 'TEXT': 'Dance Pop'}},
 'mood': {'1': {'ID': '42961', 'TEXT': 'Energizing'},
  '2': {'ID': '43056', 'TEXT': 'Edgy / Sexy'}},
 'radio_id': '',
 'review_url': '',
 

In [18]:
moods = {'Pastoral / Serene':[0,0], 'Delicate / Tranquil':[0,1], 'Hopeful / Breezy':[0,2], 'Cheerful / Playful':[0,3],
         'Carefree Pop':[0,4], 'Party / Fun':[0,5], 'Showy / Rousing':[0,6], 'Lusty / Jaunty':[0,7], 
         'Loud Celebratory':[0,8], 'Euphoric Energy':[0,9], 'Reverent / Healing':[1,0], 'Quiet / Introspective':[1,1],
         'Friendly':[1,2], 'Charming / Easygoing':[1,3], 'Soulful / Easygoing':[1,4], 'Happy / Soulful':[1,5],
         'Playful / Swingin':[1,6], 'Exuberant / Festive':[1,7], 'Upbeat Pop Groove':[1,8], 'Happy Excitement':[1,9],
         'Refined / Mannered':[2,0], 'Awakening / Stately':[2,1], 'Sweet / Sincere':[2,2], 'Heartfelt Passion':[2,3], 
         'Strong / Stable':[2,4], 'Powerful / Heroic':[2,5], 'Invigorating / Joyous':[2,6], 'Jubilant / Soulful':[2,7],
         'Ramshackle / Rollicking':[2,8], 'Wild / Rowdy':[2,9], 'Romantic / Lyrical':[3,0], 'Light Groovy':[3,1],
         'Dramatic / Romantic':[3,2], 'Lush / Romantic':[3,3], 'Dramatic Emotion':[3,4], 'Idealistic / Stirring':[3,5],
         'Focused Sparkling':[3,6], 'Triumphant / Rousing':[3,7], 'Confident / Tough':[3,8], 
         'Driving Dark Groove':[3,9], 'Tender / Sincere':[4,0], 'Gentle Bittersweet':[4,1], 'Suave / Sultry':[4,2], 
         'Dark Playful':[4,3], 'Soft Soulful':[4,4], 'Sensual Groove':[4,5], 'Dark Sparkling Lyrical':[4,6], 
         'Fiery Groove':[4,7], 'Arousing Groove':[4,8], 'Heavy Beat':[4,9], 'Lyrical Sentimental':[5,0], 
         'Cool Melancholy':[5,1], 'Intimate Bittersweet':[5,2], 'Smoky / Romantic':[5,3], 'Dreamy Pulse':[5,4], 
         'Intimate Passionate':[5,5], 'Rhythm Energetic': [5,6], 'Abstract Groove':[5,7], 'Edgy / Sexy':[5,8], 
         'Abstract Beat':[5,9], 'Mysterious / Dreamy':[6,0], 'Light Melancholy':[6,1], 'Casual Groove':[6,2], 
         'Wary / Defiant':[6,3], 'Bittersweet Pop':[6,4], 'Energetic Yearning':[6,5], 'Dark Pop':[6,6], 
         'Dark Pop Intensity':[6,7], 'Heavy Brooding':[6,8], 'Hard Positive Excitement':[6,9], 
         'Wistful / Forlorn':[7,0], 'Sad / Soulful':[7,1], 'Cool Confidence':[7,2], 'Dark Groovy':[7,3], 
         'Sensitive / Exploring':[7,4], 'Energetic Dreamy':[7,5], 'Dark Urgent':[7,6], 'Energetic Anxious':[7,7], 
         'Attitude / Defiant':[7,8], 'Hard Dark Excitement':[7,9], 'Solemn / Spiritual':[8,0], 
         'Enigmatic / Mysterious':[8,1], 'Sober / Determined':[8,2], 'Strumming Yearning':[8,3], 'Melodramatic':[8,4], 'Hypnotic Rhythm':[8,5], 'Evocative / Intriguing':[8,6], 'Energetic Melancholy':[8,7], 'Dark Hard Beat':[8,8], 'Heavy Triumphant':[8,9], 
         'Dark Cosmic':[9,0], 'Creepy / Ominous':[9,1], 'Depressed / Lonely':[9,2], 'Gritty / Soulful':[9,3], 
         'Serious / Cerebral':[9,4], 'Thrilling':[9,5], 'Dreamy Brooding':[9,6], 'Alienated / Brooding':[9,7], 
         'Chaotic / Intense':[9,8], 'Aggressive Power':[9,9]}

In [19]:
print(len(moods))

100


In [29]:
con = dbutils.get_db_handle()
querystr = "select id, name, artist_name from alltracks where name like \'%Khamaj%\'"
df = dbutils.query_db_translate_to_pandas(con, querystr)
# print(df.to_string())
print(len(df))

88


In [30]:
def chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

In [43]:
# def complete_audio_features(df):
#     features_list = []
#     for idx, row in df.iterrows():#chunks(list(df.id),50):
#         features_add = spotify.audio_features(tracks=row['id'])
#         features_list.extend(features_add)
#     return features_list

In [49]:
def complete_audio_features(stuff):
    features_list = []
    #for idx, row in df.iterrows():#chunks(list(df.id),50):
    features_add = spotify.audio_features(tracks=stuff)
    features_list.extend(features_add)
    return features_list

In [53]:
# audio_features = complete_audio_features(df)
for idx, row in df.iterrows():
    audio_features = complete_audio_features(row['id'])
    print(audio_features, "\n==========\n")
    break

[{'danceability': 0.491, 'energy': 0.529, 'key': 6, 'loudness': -8.497, 'mode': 1, 'speechiness': 0.0371, 'acousticness': 0.934, 'instrumentalness': 0.779, 'liveness': 0.0572, 'valence': 0.355, 'tempo': 131.229, 'type': 'audio_features', 'id': '4eJ8C6nGTK7G8GtlgXpfW9', 'uri': 'spotify:track:4eJ8C6nGTK7G8GtlgXpfW9', 'track_href': 'https://api.spotify.com/v1/tracks/4eJ8C6nGTK7G8GtlgXpfW9', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4eJ8C6nGTK7G8GtlgXpfW9', 'duration_ms': 1105907, 'time_signature': 3}] 

